# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def powerl(x,amp, index):
    return amp*(x**index)

def gaus(x,a,x0,sigma):
    return a*exp(-(x-x0)**2/(2*sigma**2))

def graph_identification():
    '''
    from scipy.optimize import curve_fit
    from scipy import asarray as ar,exp


    index=-1
    aver_clust=[]
    resp=[]
    for j in range(5):
    
        degrees=P1_Graphs[j].degree()
        degrees_values=sorted(set(degrees.values()))
        histogram=[list(degrees.values()).count(i)/float(nx.number_of_nodes(P1_Graphs[j])) for i in degrees_values]
        x = ar(degrees_values) 
        y = ar(histogram) 
        n = len(x)                          
        mean = sum(y)/n                 
        sigma = sum(y*(x-mean)**2)/n       

        try:            # try power law function fit
            popt_p,pcov_p = curve_fit(powerl,x,y,p0=[mean,index])
            dif=np.subtract(histogram, powerl(x, *popt_p))
            squared_sum_of_errors_powerl=sum(np.square(dif))
            #plt.figure()
            #plt.plot(x,y,'b+:',label='data')
            #plt.plot(x,powerl(x,*popt_p),'go:',label='fit powerl')
            #plt.legend()
        except:
            squared_sum_of_errors_powerl=999999

        try:            #try Gaussian function fit on the same data
            popt,pcov = curve_fit(gaus,x,y,p0=[np.mean(y),mean,sigma])
            dif=np.subtract(histogram, gaus(x, *popt))
            squared_sum_of_errors_gaus=sum(np.square(dif))
            #plt.figure()
            #plt.plot(x,y,'b+:',label='data')
            #plt.plot(x,gaus(x,*popt),'ro:',label='fit')
            #plt.legend()
        except: 
            squared_sum_of_errors_gaus=999999
        aver_clust.append(nx.average_clustering(P1_Graphs[j]))
    
        # check which fit has smaller residual errors
        if squared_sum_of_errors_gaus>=squared_sum_of_errors_powerl:
            resp.append('PA') 
        else: resp.append('SW') 
    ii=np.where(np.array(resp)=='PA')
    treshold=np.array(aver_clust)[ii].mean()*3
    jj=np.where(np.array(resp)=='SW')
    for kk in jj[0]:
        if aver_clust[kk]<treshold: resp[kk]='SW_H'
        else: resp[kk]='SW_L'
    '''
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H'] #resp

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagmentSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagmentSalary` indicates whether that person is receiving a managment position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

#print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a managment position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a managment salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a managment position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 252 with the data being the probability of receiving managment salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
def salary_predictions():
    df = pd.DataFrame(index=G.nodes())
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    sal=df.dropna()
    y_train=sal['ManagementSalary']
    X=sal[['Department','clustering', 'degree']]
    
    scaler=MinMaxScaler()
    X_train_scaled=scaler.fit_transform(X)
    X_test=df[df['ManagementSalary'].isnull()][['Department','clustering', 'degree']]
    X_test_scaled=scaler.transform(X_test)
    
    model=SVC(C=10, gamma=0.1, probability=True).fit(X_train_scaled,y_train)
    y_prob = model.predict_proba(X_test_scaled)
    yy=[np.around(i[1],decimals=1) for i in y_prob]
    return pd.Series(yy,index=df[df['ManagementSalary'].isnull()].index)

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})

(488446, 1)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.75 or higher will receive full points.

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
from sklearn.linear_model import LogisticRegression 
def new_connections_predictions():
    df=future_connections
    df['Common Neighbors'] = df.index.map(lambda id: len(list(nx.common_neighbors(G, id[0], id[1]))))
    df['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, df.index)]
    df['jaccard coefficient'] = [i[2] for i in nx.jaccard_coefficient(G, df.index)]
    df['resource_allocation'] = [i[2] for i in nx.resource_allocation_index(G, df.index)]
    df['adamic_adar'] = [i[2] for i in nx.adamic_adar_index(G, df.index)]
    sal=df.dropna()
    y=sal['Future Connection']
    X=sal[['Common Neighbors','preferential attachment','jaccard coefficient','resource_allocation','adamic_adar']]
    scaler=MinMaxScaler()
    X_train_scaled=scaler.fit_transform(X)
    X_test=df[df['Future Connection'].isnull()][['Common Neighbors','preferential attachment','jaccard coefficient','resource_allocation','adamic_adar']]
    X_test_scaled=scaler.transform(X_test)
    y_score=LogisticRegression(C=10).fit(X_train_scaled, y).predict_proba(X_test_scaled)
    yy=[np.around(i[1],decimals=1) for i in y_score]
    return pd.Series(yy,index=df[df['Future Connection'].isnull()].index)